### Auctions with RPC (Python)

An auction offering shipping container time for international trade on a major route runs on a server. Containers have a volume and are offered at a starting bid. The server allows the auction to be started and ended, all bidding items to be viewed, and bids to be placed. Bids are validated before being accepted. 

Clients have a specific budget and try to maximize the shipping capacity without overshooting the budget. For this, they can request a list of all bidding items, prepare a bid by selecting containers to bid, place multiple bids, and generate a report of the successful bids.

Clients communicate with the auction server via RPC. The server is given below. Your task is to complete the client.

**Server Side.** You can start the server on JupyterHub or your computer. If the port is in use, choose another port. The server is robust in the sense that it validates any requests and returns an error message for invalid requests.

In [12]:
from xmlrpc.server import SimpleXMLRPCServer

class AuctionServer:
    def __init__(self):
        self.items, self.bids, self.auction_live = {}, {}, False
        self.start_auction({
            'Container1': {'volume': 500, 'starting_bid': 1000},
            'Container2': {'volume': 300, 'starting_bid': 800},
            'Container3': {'volume': 450, 'starting_bid': 1200},
            'Container4': {'volume': 250, 'starting_bid': 700}
        })

    def start_auction(self, containers_info):
        if self.auction_live: return "Auction already in progress."
        self.items = {container_id: {'volume': info['volume'], 'highest_bid': info['starting_bid'], 'highest_bidder': None} for container_id, info in containers_info.items()}
        self.auction_live = True
        return "Auction started with items: " + ", ".join(containers_info)

    def place_bid(self, item, bidder, bid_amount):
        if not self.auction_live or item not in self.items: return "Auction not in progress or item not found."
        current_bid = self.items[item]['highest_bid']
        if bid_amount > current_bid:
            self.items[item]['highest_bid'] = bid_amount
            self.items[item]['highest_bidder'] = bidder
            return f"Bid placed successfully: {bidder} bid {bid_amount} on {item}."
        else:
            return f"Bid too low. Current highest bid is {current_bid}."

    def view_items(self):
        if not self.auction_live: return "Auction not in progress."
        return self.items

    def end_auction(self):
        if not self.auction_live: return "No auction to end."
        self.auction_live = False
        results = {item: self.items[item] for item in self.items}
        self.items = {}  # Reset for next auction
        return results

def run_server(port):
    with SimpleXMLRPCServer(("localhost", port), allow_none=True) as server:
        server.register_instance(AuctionServer())
        server.serve_forever()

run_server(8005)

127.0.0.1 - - [17/Dec/2023 14:04:10] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:04:43] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:05:12] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:05:12] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:05:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:05:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:05:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:05:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:06:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:06:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:06:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:06:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:07:43] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:07:43] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/2023 14:07:43] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [17/Dec/202

KeyboardInterrupt: 

**Client Side.** Complete the client! You can run the client on the same computer as the server or connect to the server remotely.

In [9]:
import xmlrpc.client

class MultiItemBiddingClient:

    def __init__(self, bidder, budget, target_volume, server_url):
        self.bidder = bidder
        self.budget = budget
        self.target_volume = target_volume
        self.server_url = server_url
        self.items_to_bid = {}
        self.acquired_volume = 0
        self.items_won = {}
        self.bids_queue = []
        
    def add_item(self, container_id, volume, max_bid):
        # adds containers to the client's bidding list
        self.items_to_bid[container_id] = {"volume": volume, "max_bid": max_bid}
    
    def prepare_bids(self):
        # determines a sequence of bids to maximize the bid values
        for container_id, info in self.items_to_bid.items():
            current_bid = 0
            while current_bid < info["max_bid"] and current_bid + 1000 <= self.budget:
                current_bid += 1000
                self.bids_queue.append((container_id, current_bid))
            
    def process_bids(self):
        # places the bids and handles the server's responses
        server_proxy = xmlrpc.client.ServerProxy(self.server_url)
        for container_id, bid in self.bids_queue:
            print(f"Attempting to bid {bid} on {container_id}")  # Debugging print
            if self.acquired_volume > self.target_volume or bid > self.budget:
                break
            response = server_proxy.place_bid(container_id, self.bidder, bid)  
            print(f"Response from server: {response}")  # Debugging print
            if "successfully" in response:
                self.budget -= bid
                self.acquired_volume += self.items_to_bid[container_id]["volume"]
                self.items_won[container_id] = bid
                
    def generate_report(self):
        # returns a list of successful bids
        return f"{self.bidder} won containers with a total volume of {self.acquired_volume} and spent {sum(self.items_won.values())}: {self.items_won}"

You can use the example below for testing.

In [ ]:
client = MultiItemBiddingClient("Your Company", 5000, 1000, "http://localhost:8005")
client.add_item('Container1', 500, 1000)  # Make sure added items exist in the Server 
client.prepare_bids()
client.process_bids()
print(client.generate_report())